Set the path to where the raw data is located.

In [3]:
import os
path_to_data = '/home/bsknight/Documents/Personal_Training_Git/Udacity/Udacity-Machine_Learning_Nanodegree/' + \
               'Capstone_Project/Data' 
os.chdir(path_to_data)

Import the necessary packages.

In [4]:
import pandas as pd
import numpy as np

Import the data.

In [5]:
df = pd.read_csv('./raw_dataset.csv', index_col='event_id')
# df = pd.read_csv('./raw_dataset_sample.csv', index_col='event_id')
len(df)

/home/bsknight/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


14686415

Recast data types.

In [6]:
df["account_id"]       = pd.to_numeric(df["account_id"] , errors='coerce')
df['account_id'].apply(str)
df["cc_date_added"]    = pd.to_datetime(df["cc_date_added"])
df["collector_tstamp"] = pd.to_datetime(df["collector_tstamp"])

**Account Filtering**

Our stated problem is to determine whether web events can be used to infer whether trial accounts ultimately convert into paying customers. Our unit of analysis is individual accounts, and so we can drop Snowplow data for vistors that are not mapped to accounts. Constraining our analysis to only trial accounts makes out data and subsequent analysis far more tractable.  

In [7]:
df = df[(pd.notnull(df["reg_date"]))]
len(df)

9757396

**Date Filtering**

We need to drop web event data that occured AFTER a credit card was added (i.e. after they agreed to become a paying customer). This filtering goes a long way towards making our data more manageable.

In [8]:
df = df[(pd.isnull(df["cc_date_added"])) | (df["cc_date_added"] > df["collector_tstamp"])]
len(df)

1946055

Drop unnecessary variables. 

In [9]:
df.drop(['reg_date','domain_sessionid','page_referrer', 'geo_region_name', 'geo_city', \
         'br_name', 'os_name', 'os_timezone', 'dvce_ismobile'], \
         inplace=True,axis=1,errors='ignore')

**URL Re-Coding**

When customers create a trial account, they receive a subdomain. This subdomain is inserted into the URL whenever somone logged onto that account visits the marketing site. For example, the URL for the pricing page for the marketing site may normally read as https://www.company_x.com/pricing/. However, if a company named 'Acme Inc.' created an account, and they visited the marketing site, then the URL would appear like https://www.company_x.com/amce_inc/pricing/. Thus, we are confronted with a challenge wherein distinct URLs do not map to distinct website experiences. To address this, I recode the URLs to omit subdomains, tags, UTM parameters, and so forth.

In [10]:
df =  df[pd.notnull(df['page_url'])]
df.loc[df['page_url'].str.contains("slbmj2010"), "page_url"] = "slbmj2010"
df.loc[df['page_url'].str.contains("10uplabs"), "page_url"] = "10uplabs"
df.loc[df['page_url'].str.contains("inventory-management-software"), "page_url"] = \
                                   "inventory-management-software"
df.loc[df['page_url'].str.contains("company_x.com/features/multichannel"), "page_url"] = \
                                   "company_x.com/features/multichannel"
df.loc[df['page_url'].str.contains("company_x.com/features/orders"), "page_url"] = \
                                   "company_x.com/features/orders"
df.loc[df['page_url'].str.contains("company_x.com/features/inventory"), "page_url"] = \
                                   "company_x.com/features/inventory"
df.loc[df['page_url'].str.contains("company_x.com/features/operations"), "page_url"] = \
                                   "company_x.com/features/operations"
df.loc[df['page_url'].str.contains("company_x.com/features/reporting"), "page_url"] = \
                                   "company_x.com/features/reporting"
df.loc[df['page_url'].str.contains("dev"), "page_url"] = "dev_page"
df.loc[df['page_url'].str.contains("preview.hs-sites.com/_hcms/preview/content"), "page_url"] = \
                                   "preview.hs-sites.com/_hcms/preview/content"
df.loc[df['page_url'].str.contains("login"), "page_url"] = "login"    
df.loc[df['page_url'].str.contains("dashboard"), "page_url"] = "dashboard"     
df.loc[df['page_url'].str.contains("lp/online-inventory-management-software-pc-s-cafe"), 
                      "page_url"] ="lp/online-inventory-management-software-pc-s-cafe"      
df.loc[df['page_url'].str.contains("signup"), "page_url"] = "signup" 
df.loc[df['page_url'].str.contains("company_x.com/magento-inventory-management"), "page_url"] = \
                                   "company_x.com/magento-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/xero-inventory-management"), "page_url"] = \
                                   "company_x.com/xero-inventory-management"                       
df.loc[df['page_url'].str.contains("company_x.com/new-user"), "page_url"] = \
                                   "company_x.com/new-user" 
df.loc[df['page_url'].str.contains("lp/drop-shipping-management-software"), "page_url"] = \
                                   "lp/drop-shipping-management-software" 
df.loc[df['page_url'].str.contains("company_x.com/quickbooks-online-inventory-management"), "page_url"] = \
                                   "company_x.com/quickbooks-online-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/signup?utm_source=capterra.com"), "page_url"] = \
                                   "company_x.com/signup?utm_source=capterra.com" 
df.loc[df['page_url'].str.contains("company_x.com/paypal-inventory-management"), "page_url"] = \
                                   "company_x.com/paypal-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/shipstation-inventory-management"), "page_url"] = \
                                   "company_x.com/shipstation-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/vend-inventory-management"), "page_url"] = \
                                   "company_x.com/vend-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/amazon-inventory-management"), "page_url"] = \
                                   "company_x.com/amazon-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/woocommerce-inventory-management"), "page_url"] = \
                                   "company_x.com/woocommerce-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/square-inventory-management"), "page_url"] = \
                                   "company_x.com/square-inventory-management" 
df.loc[df['page_url'].str.contains("company_x.com/quickbooks-inventory-management"), "page_url"] = \
                                   "company_x.com/quickbooks-inventory-management" 
df.loc[df['page_url'].str.contains("integrations/channel-listings"), "page_url"] = \
                                   "integrations/channel-listings" 
df.loc[df['page_url'].str.contains("company_x.com/etsy-inventory-tracking"), "page_url"] = \
                                   "company_x.com/etsy-inventory-tracking" 
df.loc[df['page_url'].str.contains("company_x.com/inventory/"), "page_url"] = \
                                   "company_x.com/inventory/" 
df.loc[df['page_url'].str.contains("company_x.com/integrations/manage-channels"), "page_url"] = \
                                   "company_x.com/integrations/manage-channels" 
df.loc[df['page_url'].str.contains("company_x.com/orders"), "page_url"] = \
                                   "company_x.com/orders" 
df.loc[df['page_url'].str.contains("company_x.dev/publish-listings"), "page_url"] = \
                                   "company_x.dev/publish-listings" 
df.loc[df['page_url'].str.contains("company_x.com/admin/company-info-and-locations"), "page_url"] = \
                                   "company_x.com/admin/company-info-and-locations" 
df.loc[df['page_url'].str.contains("company_x.com/admin/billing-info"), "page_url"] = \
                                   "company_x.com/admin/billing-info" 
df.loc[df['page_url'].str.contains("company_x.com/admin/contacts"), "page_url"] = \
                                   "company_x.com/admin/contacts" 
df.loc[df['page_url'].str.contains("company_x.com/purchase-orders/warehouse-transfer-orders"), "page_url"] = \
                                   "company_x.com/purchase-orders/warehouse-transfer-orders" 
df.loc[df['page_url'].str.contains("company_x.com/admin/my-profile"), "page_url"] = \
                                   "company_x.com/admin/my-profile" 
df.loc[df['page_url'].str.contains("company_x.com/admin/billing-info"), "page_url"] = \
                                   "company_x.com/admin/billing-info" 
df.loc[df['page_url'].str.contains("company_x.com/admin/my-profile"), "page_url"] = \
                                   "company_x.com/admin/my-profile" 
df.loc[df['page_url'].str.contains("company_x.com/?utm_source=shopify"), "page_url"] = \
                                   "company_x.com/?utm_source=shopify" 
df.loc[df['page_url'].str.contains("company_x.com/storenvy-inventory-management"), "page_url"] = \
                                   "company_x.com/storenvy-inventory-management"
df.loc[df['page_url'].str.contains("company_x.com/ebay-inventory-management"), "page_url"] = \
                                   "company_x.com/ebay-inventory-management"
df.loc[df['page_url'].str.contains("company_x.com/?utm_source=capterra"), "page_url"] = \
                                   "company_x.com/ebay-inventory-management"
df.loc[df['page_url'].str.contains("company_x.com/?utm_source=shopify"), "page_url"] = \
                                   "company_x.com/?utm_source=shopify"
df.loc[df['page_url'].str.contains("company_x.com/inventory"), "page_url"] = \
                                   "company_x.com/inventory"
df.loc[df['page_url'].str.contains("company_x.com/?utm_campaign=Welcome"), "page_url"] = \
                                   "company_x.com/?utm_campaign=Welcome"
df.loc[df['page_url'].str.contains("company_x.com/lp/online-inventory-management-software-pc-s-laptop"), \
                           "page_url"] = "company_x.com/lp/online-inventory-management-software-pc-s-laptop"
df.loc[df['page_url'].str.contains("company_x.com/?cachebuster"), "page_url"] = \
                                   "company_x.com/?cachebuster"
df.loc[df['page_url'].str.contains("company_x.com/lp/wb-ds?rccode=winback2015"), "page_url"] = \
                                   "company_x.com/lp/wb-ds?rccode=winback2015"
df.loc[df['page_url'].str.contains("company_x.com/bigcommerce-inventory-management"), "page_url"] = \
                                   "company_x.com/bigcommerce-inventory-management"
df.loc[df['page_url'].str.contains("company_x.com/etsy-inventory-management"), "page_url"] = \
                                   "company_x.com/etsy-inventory-management"
df.loc[df['page_url'].str.contains("lp/online-inventory-management-software-fv-int-b"), \
          "page_url"] = "lp/online-inventory-management-software-fv-int-b"
df.loc[df['page_url'].str.contains("company_x.dev/dashboard"), \
          "page_url"] = "company_x.dev/dashboard"
df.loc[df['page_url'].str.contains("company_x.com/contacts"), \
          "page_url"] = "company_x.com/contacts"
df.loc[df['page_url'].str.contains("app.hubspot.com"), "page_url"] = "app.hubspot.com"
df.loc[df['page_url'].str.contains("preview.hs-sites.com/"), "page_url"] = "preview.hs-sites.com/"
df.loc[df['page_url'].str.contains("preview.hs-sites.com/"), "page_url"] = "preview.hs-sites.com/"
df.loc[df['page_url'].str.contains("/contacts/detail/"), "page_url"] = "/contacts/detail/"
df.loc[df['page_url'].str.contains("/blog/should-you-outsource-your-order-fulfillment-process"), \
          "page_url"] = "/blog/should-you-outsource-your-order-fulfillment-process"
df.loc[df['page_url'].str.contains("codepen.io"), "page_url"] = "codepen.io"
df.loc[df['page_url'].str.contains("localhost:3000"), "page_url"] = "localhost:3000"
df.loc[df['page_url'].str.contains("kaiju"), "page_url"] = "kaiju"
df.loc[df['page_url'].str.contains("rock.slbmj2010.com"), "page_url"] = "rock.slbmj2010.com"
df.loc[df['page_url'].str.contains("ash.slbmj2010.com"), "page_url"] = "ash.slbmj2010.com"
df.loc[df['page_url'].str.contains("ultimate-guide-to-scaling-your-retail-business/managing-your-business-finances"), 
          "page_url"] = "ultimate-guide-to-scaling-your-retail-business/managing-your-business-finances"
df.loc[df['page_url'].str.contains("indinero-inventory-management-integration"), \
          "page_url"] = "indinero-inventory-management-integration"
df.loc[df['page_url'].str.contains("purchase-orders/detail"), "page_url"] = "purchase-orders/detail"
df.loc[df['page_url'].str.contains("desk.com"), "page_url"] = "desk.com"
df.loc[df['page_url'].str.contains("ghostinspector"), "page_url"] = "ghostinspector"
df.loc[df['page_url'].str.contains("resources/customer-stories/bedface"), \
          "page_url"] = "resources/customer-stories/bedface"
df.loc[df['page_url'].str.contains("scout-3pl-inventory-management"), \
          "page_url"] = "scout-3pl-inventory-management"
df.loc[df['page_url'].str.contains("resource/reports/when-to-outsource-retail-operations-to-a-3pl-provider"), \
          "page_url"] = "resource/reports/when-to-outsource-retail-operations-to-a-3pl-provider"
df.loc[df['page_url'].str.contains("resources/customer-stories/chubbies-2"), \
          "page_url"] = "resources/customer-stories/chubbies-2"
df.loc[df['page_url'].str.contains("why-stitch"), \
          "page_url"] = "why-stitch"
df.loc[df['page_url'].str.contains("integrations/newgistics"), \
          "page_url"] = "integrations/newgistics"
df.loc[df['page_url'].str.contains("blog/stitch-product-updates-july"), \
          "page_url"] = "blog/stitch-product-updates-july"
df.loc[df['page_url'].str.contains("blog/topic/stitch-updates"), \
          "page_url"] = "blog/topic/stitch-updates"
df.loc[df['page_url'].str.contains("blog/topic/inventory-management"), \
          "page_url"] = "blog/topic/inventory-management"
df.loc[df['page_url'].str.contains("blog/topic/customer-spotlight"), \
          "page_url"] = "blog/topic/customer-spotlight"
df.loc[df['page_url'].str.contains("blog/topic/master-your-data"), \
          "page_url"] = "blog/topic/master-your-data"
df.loc[df['page_url'].str.contains("lp/inventory-accounting-software"), \
          "page_url"] = "lp/inventory-accounting-software"
df.loc[df['page_url'].str.contains("blog/all"), "page_url"] = "blog/all"
df.loc[df['page_url'].str.contains("new-user?signup"), "page_url"] = "new-user?signup"
df.loc[df['page_url'].str.contains("blog/why-fba-sellers-need-amazon-repricing-tools"), \
          "page_url"] = "blog/why-fba-sellers-need-amazon-repricing-tools"
df.loc[df['page_url'].str.contains("blog/topic/inventory-setups"), "page_url"] = "blog/topic/inventory-setups"
df.loc[df['page_url'].str.contains("blog/topic/multichannel-operations"), \
          "page_url"] = "blog/topic/multichannel-operations"
df.loc[df['page_url'].str.contains("blog/black-fridays-now"), \
          "page_url"] = "blog/black-fridays-now"
df.loc[df['page_url'].str.contains("scout-inventory-management"), "page_url"] = "scout-inventory-management"
df.loc[df['page_url'].str.contains("chapter-1.html"),  "page_url"] = "chapter-1.html"
df.loc[df['page_url'].str.contains("chapter-2.html"),  "page_url"] = "chapter-2.html"
df.loc[df['page_url'].str.contains("chapter-3.html"),  "page_url"] = "chapter-3.html"
df.loc[df['page_url'].str.contains("chapter-4.html"),  "page_url"] = "chapter-4.html"
df.loc[df['page_url'].str.contains("chapter-5.html"),  "page_url"] = "chapter-5.html"
df.loc[df['page_url'].str.contains("chapter-6.html"),  "page_url"] = "chapter-6.html"
df.loc[df['page_url'].str.contains("chapter-7.html"),  "page_url"] = "chapter-7.html"
df.loc[df['page_url'].str.contains("chapter-8.html"),  "page_url"] = "chapter-8.html"
df.loc[df['page_url'].str.contains("chapter-9.html"),  "page_url"] = "chapter-9.html"
df.loc[df['page_url'].str.contains("chapter-10.html"), "page_url"] = "chapter-10.html"
df.loc[df['page_url'].str.contains("resource/webinars"), "page_url"] = "resource/webinars"
df.loc[df['page_url'].str.contains("resource/reports/preparing-2016-retail-holiday-data-report"), \
          "page_url"] = "resource/reports/preparing-2016-retail-holiday-data-report"
df.loc[df['page_url'].str.contains("integrations/spscommerce-edi-inventory-management"), \
          "page_url"] = "integrations/spscommerce-edi-inventory-management"
df.loc[df['page_url'].str.contains("integration-category/marketplaces"), \
          "page_url"] = "integration-category/marketplaces"
df.loc[df['page_url'].str.contains("request-demo"), "page_url"] = "request-demo"
df.loc[df['page_url'].str.contains("pricing-p7.2"), "page_url"] = "pricing-p7.2"
df.loc[df['page_url'].str.contains("pricing-7.1"), "page_url"] = "pricing-7.1"
df.loc[df['page_url'].str.contains("pricing-feature-detail-7.1"), "page_url"] = "pricing-feature-detail-7.1"
df.loc[df['page_url'].str.contains("resources/customer-stories/nomad"), \
          "page_url"] = "resources/customer-stories/nomad"
df.loc[df['page_url'].str.contains("integration-category/erp/"), \
          "page_url"] = "integration-category/erp/"
df.loc[df['page_url'].str.contains("pricing/"), \
          "page_url"] = "pricing/"
df.loc[df['page_url'].str.contains("contact-us/"), \
          "page_url"] = "contact-us/"
df.loc[df['page_url'].str.contains("about/careers/"), \
          "page_url"] = "about/careers/"
df.loc[df['page_url'].str.contains("about/"), \
          "page_url"] = "about/"
df.loc[df['page_url'].str.contains("aboutus"), \
          "page_url"] = "aboutus"
df.loc[df['page_url'].str.contains("integrations/lightspeed-retail"), \
          "page_url"] = "integrations/lightspeed-retail"
df.loc[df['page_url'].str.contains("resources/apparel/chubbies-2"), \
          "page_url"] = "resources/apparel/chubbies-2"
df.loc[df['page_url'].str.contains("integrations/bigcommerce/"), \
          "page_url"] = "integrations/bigcommerce/"
df.loc[df['page_url'].str.contains(
"press/stitch-labs-announces-partnership-to-streamline-inventory-and-operations-for-high-growth-retailer"), \
"page_url"] = \
"press/stitch-labs-announces-partnership-to-streamline-inventory-and-operations-for-high-growth-retailer"
df.loc[df['page_url'].str.contains("integrations/scout/"), \
          "page_url"] = "integrations/scout/"
df.loc[df['page_url'].str.contains("homepage.html"), \
          "page_url"] = "homepage.html"
df.loc[df['page_url'].str.contains("retail-amplifier/"), \
          "page_url"] = "retail-amplifier/"
df.loc[df['page_url'].str.contains("integration-category/test"), \
          "page_url"] = "integration-category/test"
df.loc[df['page_url'].str.contains("getapp.com/x/stitch-labs-application"), \
          "page_url"] = "getapp.com/x/stitch-labs-application"
df.loc[df['page_url'].str.contains("pricing-feature-detail.html"), \
          "page_url"] = "pricing-feature-detail.html"
df.loc[df['page_url'].str.contains("hs/manage-preferences/unsubscribe-all"), \
          "page_url"] = "hs/manage-preferences/unsubscribe-all"
df.loc[df['page_url'].str.contains("inventory-control/"), \
          "page_url"] = "inventory-control/"
df.loc[df['page_url'].str.contains("integration-category/edi"), \
          "page_url"] = "integration-category/edi"
df.loc[df['page_url'].str.contains("integration-category/edi"), \
          "page_url"] = "integration-category/edi"

In [11]:
df.loc[df['page_url'].str.contains("company_x.com") & 
~df["page_url"].str.contains("slbmj2010") &
~df["page_url"].str.contains("10uplabs") &
~df["page_url"].str.contains("company_x.com/inventory-management-software") &
~df["page_url"].str.contains("company_x.com/features/multichannel") &
~df["page_url"].str.contains("company_x.com/features/operations") &
~df["page_url"].str.contains("company_x.com/features/orders") &
~df["page_url"].str.contains("company_x.com/features/inventory") &
~df["page_url"].str.contains("company_x.com/features/operations") &
~df["page_url"].str.contains("company_x.com/features/reporting") &
~df["page_url"].str.contains("dev_page") &
~df["page_url"].str.contains("preview.hs-sites.com/_hcms/preview/content") &
~df["page_url"].str.contains("company_x.com/login") &    
~df["page_url"].str.contains("company_x.com/dashboard") &     
~df["page_url"].str.contains("lp/online-inventory-management-software-pc-s-cafe") &      
~df["page_url"].str.contains("company_x.com/signup") & 
~df["page_url"].str.contains("company_x.com/magento-inventory-management") & 
~df["page_url"].str.contains("company_x.com/xero-inventory-management") &                       
~df["page_url"].str.contains("company_x.com/new-user") & 
~df["page_url"].str.contains("lp/drop-shipping-management-software") & 
~df["page_url"].str.contains("company_x.com/quickbooks-online-inventory-management") & 
~df["page_url"].str.contains("company_x.com/signup?utm_source=capterra.com") & 
~df["page_url"].str.contains("company_x.com/paypal-inventory-management") & 
~df["page_url"].str.contains("company_x.com/shipstation-inventory-management") & 
~df["page_url"].str.contains("company_x.com/vend-inventory-management") & 
~df["page_url"].str.contains("company_x.com/amazon-inventory-management") & 
~df["page_url"].str.contains("company_x.com/woocommerce-inventory-management") & 
~df["page_url"].str.contains("company_x.com/square-inventory-management") & 
~df["page_url"].str.contains("company_x.com/quickbooks-inventory-management") & 
~df["page_url"].str.contains("integrations/channel-listings") & 
~df["page_url"].str.contains("company_x.com/etsy-inventory-tracking") & 
~df["page_url"].str.contains("company_x.com/inventory/") & 
~df["page_url"].str.contains("company_x.com/integrations/manage-channels") & 
~df["page_url"].str.contains("company_x.com/orders") & 
~df["page_url"].str.contains("company_x.dev/publish-listings") & 
~df["page_url"].str.contains("company_x.com/admin/company-info-and-locations") & 
~df["page_url"].str.contains("company_x.com/admin/billing-info") & 
~df["page_url"].str.contains("company_x.com/admin/contacts") & 
~df["page_url"].str.contains("company_x.com/purchase-orders/warehouse-transfer-orders") & 
~df["page_url"].str.contains("company_x.com/admin/my-profile") & 
~df["page_url"].str.contains("company_x.com/admin/billing-info") & 
~df["page_url"].str.contains("company_x.com/admin/my-profile") & 
~df["page_url"].str.contains("company_x.com/?utm_source=shopify") & 
~df["page_url"].str.contains("company_x.com/storenvy-inventory-management") &
~df["page_url"].str.contains("company_x.com/ebay-inventory-management") &
~df["page_url"].str.contains("company_x.com/ebay-inventory-management") &
~df["page_url"].str.contains("company_x.com/?utm_source=shopify") &
~df["page_url"].str.contains("company_x.com/inventory") &
~df["page_url"].str.contains("company_x.com/?utm_campaign=Welcome") &
~df["page_url"].str.contains("company_x.com/lp/online-inventory-management-software-pc-s-laptop") &
~df["page_url"].str.contains("company_x.com/?cachebuster") &
~df["page_url"].str.contains("company_x.com/lp/wb-ds?rccode=winback2015") &
~df["page_url"].str.contains("company_x.com/bigcommerce-inventory-management") &
~df["page_url"].str.contains("company_x.com/etsy-inventory-management") &
~df["page_url"].str.contains("lp/online-inventory-management-software-fv-int-b") &
~df["page_url"].str.contains("company_x.dev/dashboard") &
~df["page_url"].str.contains("company_x.com/contacts") &
~df["page_url"].str.contains("app.hubspot.com") &
~df["page_url"].str.contains("preview.hs-sites.com/") &
~df["page_url"].str.contains("preview.hs-sites.com/") &
~df["page_url"].str.contains("/contacts/detail/") &
~df["page_url"].str.contains("/blog/should-you-outsource-your-order-fulfillment-process") &
~df["page_url"].str.contains("codepen.io") &
~df["page_url"].str.contains("localhost:3000") &
~df["page_url"].str.contains("kaiju") &
~df["page_url"].str.contains("rock.slbmj2010.com") &
~df["page_url"].str.contains("ash.slbmj2010.com") &
~df["page_url"].str.contains("ultimate-guide-to-scaling-your-retail-business/managing-your-business-finances") &
~df["page_url"].str.contains("indinero-inventory-management-integration") &
~df["page_url"].str.contains("purchase-orders/detail") &
~df["page_url"].str.contains("desk.com") &
~df["page_url"].str.contains("ghostinspector") &
~df["page_url"].str.contains("resources/customer-stories/bedface") &
~df["page_url"].str.contains("scout-3pl-inventory-management") &
~df["page_url"].str.contains("resource/reports/when-to-outsource-retail-operations-to-a-3pl-provider") &
~df["page_url"].str.contains("resources/customer-stories/chubbies-2") &
~df["page_url"].str.contains("why-stitch") &
~df["page_url"].str.contains("integrations/newgistics") &
~df["page_url"].str.contains("blog/stitch-product-updates-july") &
~df["page_url"].str.contains("blog/topic/stitch-updates") &
~df["page_url"].str.contains("blog/topic/inventory-management") &
~df["page_url"].str.contains("blog/topic/customer-spotlight") &
~df["page_url"].str.contains("blog/topic/master-your-data") &
~df["page_url"].str.contains("lp/inventory-accounting-software") &
~df["page_url"].str.contains("blog/all") &
~df["page_url"].str.contains("new-user?signup") &
~df["page_url"].str.contains("blog/why-fba-sellers-need-amazon-repricing-tools") &
~df["page_url"].str.contains("blog/topic/inventory-setups") &
~df["page_url"].str.contains("blog/topic/multichannel-operations") &
~df["page_url"].str.contains("blog/black-fridays-now") &
~df["page_url"].str.contains("scout-inventory-management") &
~df["page_url"].str.contains("chapter-1.html") &
~df["page_url"].str.contains("chapter-2.html") &
~df["page_url"].str.contains("chapter-3.html") &
~df["page_url"].str.contains("chapter-4.html") &
~df["page_url"].str.contains("chapter-5.html") &
~df["page_url"].str.contains("chapter-6.html") &
~df["page_url"].str.contains("chapter-7.html") &
~df["page_url"].str.contains("chapter-8.html") &
~df["page_url"].str.contains("chapter-9.html") &
~df["page_url"].str.contains("chapter-10.html") &
~df["page_url"].str.contains("resource/webinars") &
~df["page_url"].str.contains("resource/reports/preparing-2016-retail-holiday-data-report") &
~df["page_url"].str.contains("integrations/spscommerce-edi-inventory-management") &
~df["page_url"].str.contains("integration-category/marketplaces") &
~df["page_url"].str.contains("request-demo") &
~df["page_url"].str.contains("pricing-p7.2") &
~df["page_url"].str.contains("pricing-7.1") &
~df["page_url"].str.contains("pricing-feature-detail-7.1") &
~df["page_url"].str.contains("resources/customer-stories/nomad") &
~df["page_url"].str.contains("integration-category/erp/") &
~df["page_url"].str.contains("pricing/") &
~df["page_url"].str.contains("contact-us/") &
~df["page_url"].str.contains("about/careers/") &
~df["page_url"].str.contains("about/") &
~df["page_url"].str.contains("aboutus") &
~df["page_url"].str.contains("integrations/lightspeed-retail") &
~df["page_url"].str.contains("resources/apparel/chubbies-2") &
~df["page_url"].str.contains("integrations/bigcommerce/") &
~df["page_url"].str.contains(
"press/stitch-labs-announces-partnership-to-streamline-inventory-and-operations-for-high-growth-retailer") &
~df["page_url"].str.contains("integrations/scout/") &
~df["page_url"].str.contains("homepage.html") &
~df["page_url"].str.contains("retail-amplifier/") &
~df["page_url"].str.contains("integration-category/test") &
~df["page_url"].str.contains("getapp.com/x/stitch-labs-application") &
~df["page_url"].str.contains("pricing-feature-detail.html") &
~df["page_url"].str.contains("hs/manage-preferences/unsubscribe-all") &
~df["page_url"].str.contains("inventory-control/") &
~df["page_url"].str.contains("integration-category/edi") &
~df["page_url"].str.contains("/orders"),
"page_url"] = "company_x.com"

Here I filter out events that do not take place within the 100 URLs specified above. 

In [12]:
a = [
'company_x.com/inventory',
'company_x.com/features/multichannel',
 'company_x.com/features/orders',
 'company_x.com/features/inventory',
 'company_x.com/features/operations',
 'preview.hs-sites.com/',
 'company_x.com/features/reporting',
 'dev_page',
 'app.hubspot.com',
 'company_x.com',
 'login',
 'dashboard',
 'company_x.com/new-user',
 'company_x.com/woocommerce-inventory-management',
 'company_x.com/amazon-inventory-management',
 'lp/drop-shipping-management-software',
 'signup',
 'inventory-management-software',
 'company_x.com/etsy-inventory-tracking',
 'company_x.com/storenvy-inventory-management',
 'company_x.com/ebay-inventory-management',
 'company_x.com/quickbooks-inventory-management',
 'company_x.com/square-inventory-management',
 'company_x.com/xero-inventory-management',
 'company_x.com/magento-inventory-management',
 'company_x.com/quickbooks-online-inventory-management',
 'company_x.com/integrations/manage-channels',
 'company_x.com/paypal-inventory-management',
 'company_x.com/orders',
 'integrations/channel-listings',
 'company_x.com/shipstation-inventory-management',
 'company_x.com/vend-inventory-management',
 'company_x.com/bigcommerce-inventory-management',
 'company_x.com/etsy-inventory-management',
 'slbmj2010',
 'company_x.com/admin/my-profile',
 'company_x.com/admin/billing-info',
 'localhost:3000',
 'company_x.com/contacts',
 'company_x.com/admin/company-info-and-locations',
 'company_x.com/purchase-orders/warehouse-transfer-orders',
 'about/',
 'blog/topic/master-your-data',
 'blog/topic/inventory-management',
 'blog/topic/customer-spotlight',
 'blog/all',
 'blog/topic/multichannel-operations',
 'blog/topic/stitch-updates',
 'blog/topic/inventory-setups',
 'about:blank',
 'https://app.t3designs.com/orders',
 'retail-amplifier/',
 'desk.com',
 'request-demo',
 'getapp.com/x/stitch-labs-application',
 'codepen.io',
 'scout-inventory-management',
 'chapter-1.html',
 'chapter-2.html',
 'pricing-feature-detail.html',
 'blog/stitch-product-updates-july',
 'chapter-4.html',
 'chapter-6.html',
 'chapter-5.html',
 'chapter-10.html',
 'chapter-7.html',
 'chapter-3.html',
 'chapter-9.html',
 'purchase-orders/detail',
 'chapter-8.html',
 'pricing-7.1',
 'hs/manage-preferences/unsubscribe-all',
 'homepage.html',
 'indinero-inventory-management-integration',
 '/blog/should-you-outsource-your-order-fulfillment-process',
 'ultimate-guide-to-scaling-your-retail-business/managing-your-business-finances',
 'blog/why-fba-sellers-need-amazon-repricing-tools',
 'lp/inventory-accounting-software',
 'press/stitch-labs-announces-partnership-to-streamline-inventory-and-operations-for-high-growth-retailer',
 'pricing/',
 'why-stitch',
 'integration-category/marketplaces',
 'resource/reports/preparing-2016-retail-holiday-data-report',
 'contact-us/',
 'resource/webinars',
 'integrations/lightspeed-retail',
 'integration-category/erp/',
 'integrations/spscommerce-edi-inventory-management',
 'integrations/newgistics',
 'resource/reports/when-to-outsource-retail-operations-to-a-3pl-provider',
 'integrations/scout/',
 'integrations/bigcommerce/',
 'ghostinspector',
 'blog/black-fridays-now',
 'https://company_x.global.ssl.fastly.net/',
 'https://stitch-labs.local/integration-category/ecommerce-platforms/',
 '10uplabs',
 'https://stitch-labs.local/test-page/',
 'https://stitch-labs.local/',
 'https://stitch-labs.local/integration-category/other-integrations']
len(a)

100

In [13]:
df = df[df["page_url"].isin(a)]

**Event Aggregation**

Snowplow utilizes six different types of events ['page_view', 'event', 'link_click', 'change_form', 'submit_form', 'page_ping']. The following code block counts the number of events by event type / page url by account ID. The first two lines create a series of dummy (boolean) variable for each event type. We can then sum these ones and zeros by page URL and account ID to assess how many of each kind of event happened where for each account.

In [14]:
df_event_name = pd.get_dummies(df['event_name'])  
df = pd.concat([df, df_event_name], axis=1)
###################################################################################################################
page_view = pd.DataFrame(df.groupby(['account_id', 'page_url'])['page_view'].sum())
page_view = page_view.reset_index()
page_view = page_view.pivot_table(index='account_id', columns='page_url', values='page_view')
page_view.columns = [str(col) + 'page_view' for col in page_view.columns]
page_view.fillna(value=0, inplace=True)
###################################################################################################################
event = pd.DataFrame(df.groupby(['account_id', 'page_url'])['event'].sum())
event = event.reset_index()
event = event.pivot_table(index='account_id', columns='page_url', values='event')
event.columns = [str(col) + 'event' for col in event.columns]
event.fillna(value=0, inplace=True)
###################################################################################################################
link_click = pd.DataFrame(df.groupby(['account_id', 'page_url'])['link_click'].sum())
link_click = link_click.reset_index()
link_click = link_click.pivot_table(index='account_id', columns='page_url', values='link_click')
link_click.columns = [str(col) + 'link_click' for col in link_click.columns]
link_click.fillna(value=0, inplace=True)
###################################################################################################################
change_form = pd.DataFrame(df.groupby(['account_id', 'page_url'])['change_form'].sum())
change_form = change_form.reset_index()
change_form = change_form.pivot_table(index='account_id', columns='page_url', values='change_form')
change_form.columns = [str(col) + 'change_form' for col in change_form.columns]
change_form.fillna(value=0, inplace=True)
###################################################################################################################
submit_form = pd.DataFrame(df.groupby(['account_id', 'page_url'])['submit_form'].sum())
submit_form = submit_form.reset_index()
submit_form = submit_form.pivot_table(index='account_id', columns='page_url', values='submit_form')
submit_form.columns = [str(col) + 'submit_form' for col in submit_form.columns]
submit_form.fillna(value=0, inplace=True)
###################################################################################################################
page_ping = pd.DataFrame(df.groupby(['account_id', 'page_url'])['page_ping'].sum())
page_ping = page_ping.reset_index()
page_ping = page_ping.pivot_table(index='account_id', columns='page_url', values='page_ping')
page_ping.columns = [str(col) + 'page_ping' for col in page_ping.columns]
page_ping.fillna(value=0, inplace=True)
###################################################################################################################
df.drop(['change_form', 'event', 'link_click', 'page_ping', 'page_view', 
         'submit_form', 'collector_tstamp'], inplace=True, axis=1, errors='ignore')

We take our counts of events/URLs and create a new data frame to contain them

In [15]:
event_df = pd.concat([page_view, event, link_click, change_form, submit_form, page_ping], axis=1)
lst = [page_view, event, link_click, change_form, submit_form, page_ping]
cols = list(event_df.columns.values)
event_df[cols] = event_df[cols].applymap(np.int64)
del lst   

The next bit of information we want is the number of unique cookies (i.e. 'domain_userid) associated with each account. Theoretically, more cookies might corelate with more people whihc might correlate to more overall interest in the product and more likelihood of becoming a paying customer.

In [16]:
cookies = pd.DataFrame(df.groupby('account_id').domain_userid.nunique())
cookies = cookies.rename(columns={'domain_userid': 'distinct_cookies'})
event_df = pd.concat([event_df, cookies], axis=1)
lst = [cookies]
del lst 

Now we are counting the number of distinct sessions by account. This information is contained in the 'domain_sessionidx' variable. The count of sessions per cookie is cumulative, so we must first select the maximum value of 'domain_sessionidx' by 'domain_userid', then sum the results by 'account_id.'

In [17]:
sessions = pd.DataFrame(df.groupby(['account_id','domain_userid']).domain_sessionidx.max())
sessions.swaplevel(0, 1, axis=0)
sessions = sessions.reset_index()
sessions.drop('domain_userid', axis=1, inplace=True)
sessions = pd.DataFrame(sessions.groupby('account_id').domain_sessionidx.sum())
sessions = sessions.rename(columns={'domain_sessionidx': 'distinct_sessions'})
event_df = pd.concat([event_df, sessions], axis=1)
lst = [sessions]
del lst 

The next feature we are creating is the number of page views originating from each country. Here I subset the event data to only include page views, then aggregate by country.

In [18]:
page_views = df[df['event_name'] == 'page_view']
page_views = page_views.set_index(['account_id'])
page_views = pd.get_dummies(page_views['geo_country']) 

In [19]:
A1 = pd.DataFrame(page_views.groupby(page_views.index).A1.sum())
A1.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, A1], axis=1)
lst = [A1]
del lst 
#############################################################
A2 = pd.DataFrame(page_views.groupby(page_views.index).A2.sum())
A2.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, A2], axis=1)
lst = [A2]
del lst 
#############################################################
AE = pd.DataFrame(page_views.groupby(page_views.index).AE.sum())
AE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AE], axis=1)
lst = [AE]
del lst 
#############################################################
AF = pd.DataFrame(page_views.groupby(page_views.index).AF.sum())
AF.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AF], axis=1)
lst = [AF]
del lst 
#############################################################
AG = pd.DataFrame(page_views.groupby(page_views.index).AG.sum())
AG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AG], axis=1)
lst = [AG]
del lst 
#############################################################
AL = pd.DataFrame(page_views.groupby(page_views.index).AL.sum())
AL.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AL], axis=1)
lst = [AL]
del lst 
#############################################################
AM = pd.DataFrame(page_views.groupby(page_views.index).AM.sum())
AM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AM], axis=1)
lst = [AM]
del lst 
#############################################################
AP = pd.DataFrame(page_views.groupby(page_views.index).AP.sum())
AP.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AP], axis=1)
lst = [AP]
del lst 
#############################################################
AR = pd.DataFrame(page_views.groupby(page_views.index).AR.sum())
AR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AR], axis=1)
lst = [AR]
del lst 
#############################################################
AT = pd.DataFrame(page_views.groupby(page_views.index).AT.sum())
AT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AT], axis=1)
lst = [AT]
del lst 
#############################################################
AU = pd.DataFrame(page_views.groupby(page_views.index).AU.sum())
AU.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AU], axis=1)
lst = [AU]
del lst 
#############################################################
AZ = pd.DataFrame(page_views.groupby(page_views.index).AZ.sum())
AZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, AZ], axis=1)
lst = [AZ]
del lst 
#############################################################
BA = pd.DataFrame(page_views.groupby(page_views.index).BA.sum())
BA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BA], axis=1)
lst = [BA]
del lst 
#############################################################
BB = pd.DataFrame(page_views.groupby(page_views.index).BB.sum())
BB.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BB], axis=1)
lst = [BB]
del lst 
#############################################################
BD = pd.DataFrame(page_views.groupby(page_views.index).BD.sum())
BD.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BD], axis=1)
lst = [BD]
del lst 
#############################################################
BE = pd.DataFrame(page_views.groupby(page_views.index).BE.sum())
BE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BE], axis=1)
lst = [BE]
del lst 
#############################################################
BG = pd.DataFrame(page_views.groupby(page_views.index).BG.sum())
BG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BG], axis=1)
lst = [BG]
del lst 
#############################################################
BH = pd.DataFrame(page_views.groupby(page_views.index).BH.sum())
BH.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BH], axis=1)
lst = [BH]
del lst 
#############################################################
BN = pd.DataFrame(page_views.groupby(page_views.index).BN.sum())
BN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BN], axis=1)
lst = [BN]
del lst 
#############################################################
BR = pd.DataFrame(page_views.groupby(page_views.index).BR.sum())
BR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BR], axis=1)
lst = [BR]
del lst 
#############################################################
BS = pd.DataFrame(page_views.groupby(page_views.index).BS.sum())
BS.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BS], axis=1)
lst = [BS]
del lst 
#############################################################
BW = pd.DataFrame(page_views.groupby(page_views.index).BW.sum())
BW.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BW], axis=1)
lst = [BW]
del lst 
#############################################################
BY = pd.DataFrame(page_views.groupby(page_views.index).BY.sum())
BY.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BY], axis=1)
lst = [BY]
del lst 
#############################################################
BZ = pd.DataFrame(page_views.groupby(page_views.index).BZ.sum())
BZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, BZ], axis=1)
lst = [BZ]
del lst 
#############################################################
CA = pd.DataFrame(page_views.groupby(page_views.index).CA.sum())
CA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CA], axis=1)
lst = [BR]
del lst 
#############################################################
CH = pd.DataFrame(page_views.groupby(page_views.index).CH.sum())
CH.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CH], axis=1)
lst = [CH]
del lst 
#############################################################
CI = pd.DataFrame(page_views.groupby(page_views.index).CI.sum())
CI.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CI], axis=1)
lst = [CI]
del lst 
#############################################################
CL = pd.DataFrame(page_views.groupby(page_views.index).CL.sum())
CL.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CL], axis=1)
lst = [CL]
del lst 
#############################################################
CM = pd.DataFrame(page_views.groupby(page_views.index).CM.sum())
CM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CM], axis=1)
lst = [CM]
del lst 
#############################################################
CN = pd.DataFrame(page_views.groupby(page_views.index).CN.sum())
CN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CN], axis=1)
lst = [CN]
del lst 
#############################################################
CO = pd.DataFrame(page_views.groupby(page_views.index).CO.sum())
CO.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CO], axis=1)
lst = [CO]
del lst 
#############################################################
CR = pd.DataFrame(page_views.groupby(page_views.index).CR.sum())
CR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CR], axis=1)
lst = [CR]
del lst 
#############################################################
CY= pd.DataFrame(page_views.groupby(page_views.index).CY.sum())
CY.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CY], axis=1)
lst = [CY]
del lst 
#############################################################
CZ = pd.DataFrame(page_views.groupby(page_views.index).CZ.sum())
CZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CZ], axis=1)
lst = [CZ]
del lst 
#############################################################
DE = pd.DataFrame(page_views.groupby(page_views.index).DE.sum())
DE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, DE], axis=1)
lst = [DE]
del lst 
#############################################################
DK = pd.DataFrame(page_views.groupby(page_views.index).DK.sum())
DK.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, DK], axis=1)
lst = [DK]
del lst 
#############################################################
DO = pd.DataFrame(page_views.groupby(page_views.index).DO.sum())
DO.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, DO], axis=1)
lst = [DO]
del lst 
#############################################################
DZ = pd.DataFrame(page_views.groupby(page_views.index).DZ.sum())
DZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, DZ], axis=1)
lst = [DZ]
del lst 
#############################################################
EC = pd.DataFrame(page_views.groupby(page_views.index).EC.sum())
EC.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, EC], axis=1)
lst = [EC]
del lst 
#############################################################
EE = pd.DataFrame(page_views.groupby(page_views.index).EE.sum())
EE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, EE], axis=1)
lst = [EE]
del lst
#############################################################
EG = pd.DataFrame(page_views.groupby(page_views.index).EG.sum())
EG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, EG], axis=1)
lst = [EG]
del lst 
#############################################################
ES = pd.DataFrame(page_views.groupby(page_views.index).ES.sum())
ES.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ES], axis=1)
lst = [ES]
del lst 
#############################################################
ET = pd.DataFrame(page_views.groupby(page_views.index).ET.sum())
ET.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ET], axis=1)
lst = [ET]
del lst 
#############################################################
EU = pd.DataFrame(page_views.groupby(page_views.index).EU.sum())
EU.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, EU], axis=1)
lst = [EU]
del lst 
#############################################################
FI = pd.DataFrame(page_views.groupby(page_views.index).FI.sum())
FI.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, FI], axis=1)
lst = [FI]
del lst 
#############################################################
FJ = pd.DataFrame(page_views.groupby(page_views.index).FJ.sum())
FJ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, FJ], axis=1)
lst = [FJ]
del lst 
#############################################################
FR = pd.DataFrame(page_views.groupby(page_views.index).FR.sum())
FR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, FR], axis=1)
lst = [FR]
del lst 
#############################################################
GB = pd.DataFrame(page_views.groupby(page_views.index).GB.sum())
GB.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GB], axis=1)
lst = [GB]
del lst 
#############################################################
GD = pd.DataFrame(page_views.groupby(page_views.index).GD.sum())
GD.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GD], axis=1)
lst = [GD]
del lst 
#############################################################
GE = pd.DataFrame(page_views.groupby(page_views.index).GE.sum())
GE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GE], axis=1)
lst = [GE]
del lst 
#############################################################
GG = pd.DataFrame(page_views.groupby(page_views.index).GG.sum())
GG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GG], axis=1)
lst = [GG]
del lst 
#############################################################
GH = pd.DataFrame(page_views.groupby(page_views.index).GH.sum())
GH.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GH], axis=1)
lst = [GH]
del lst 
#############################################################
GI = pd.DataFrame(page_views.groupby(page_views.index).GI.sum())
GI.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GI], axis=1)
lst = [GI]
del lst 
#############################################################
GM = pd.DataFrame(page_views.groupby(page_views.index).GM.sum())
GM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GM], axis=1)
lst = [GM]
del lst 
#############################################################
GR = pd.DataFrame(page_views.groupby(page_views.index).GR.sum())
GR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GR], axis=1)
lst = [GR]
del lst 
#############################################################
GT = pd.DataFrame(page_views.groupby(page_views.index).GT.sum())
GT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, GT], axis=1)
lst = [GT]
del lst 
#############################################################
HK = pd.DataFrame(page_views.groupby(page_views.index).HK.sum())
HK.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, HK], axis=1)
lst = [HK]
del lst 
#############################################################
HN = pd.DataFrame(page_views.groupby(page_views.index).HN.sum())
HN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, HN], axis=1)
lst = [HN]
del lst 
#############################################################
HR = pd.DataFrame(page_views.groupby(page_views.index).HR.sum())
HR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, HR], axis=1)
lst = [HR]
del lst 
#############################################################
HT = pd.DataFrame(page_views.groupby(page_views.index).HT.sum())
HT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, HT], axis=1)
lst = [HT]
del lst 
#############################################################
HU = pd.DataFrame(page_views.groupby(page_views.index).HU.sum())
HU.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, HU], axis=1)
lst = [HU]
del lst 
#############################################################
ID = pd.DataFrame(page_views.groupby(page_views.index).ID.sum())
ID.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ID], axis=1)
lst = [ID]
del lst 
#############################################################
IE = pd.DataFrame(page_views.groupby(page_views.index).IE.sum())
IE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IE], axis=1)
lst = [IE]
del lst 
#############################################################
IL = pd.DataFrame(page_views.groupby(page_views.index).IL.sum())
IL.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IL], axis=1)
lst = [IL]
del lst 
#############################################################
IN = pd.DataFrame(page_views.groupby(page_views.index).IN.sum())
IN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IN], axis=1)
lst = [IN]
del lst 
#############################################################
IQ = pd.DataFrame(page_views.groupby(page_views.index).IQ.sum())
IQ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IQ], axis=1)
lst = [IQ]
del lst 
#############################################################
IR = pd.DataFrame(page_views.groupby(page_views.index).IR.sum())
IR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IR], axis=1)
lst = [IR]
del lst 
#############################################################
IS = pd.DataFrame(page_views.groupby(page_views.index).IS.sum())
IS.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IS], axis=1)
lst = [IS]
del lst 
#############################################################
IT = pd.DataFrame(page_views.groupby(page_views.index).IT.sum())
IT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IT], axis=1)
lst = [IT]
del lst 
#############################################################
JE = pd.DataFrame(page_views.groupby(page_views.index).JE.sum())
JE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, JE], axis=1)
lst = [JE]
del lst 
#############################################################
JM = pd.DataFrame(page_views.groupby(page_views.index).JM.sum())
JM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, JM], axis=1)
lst = [JM]
del lst 
#############################################################
JO = pd.DataFrame(page_views.groupby(page_views.index).JO.sum())
JO.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, JO], axis=1)
lst = [JO]
del lst 
#############################################################
JP = pd.DataFrame(page_views.groupby(page_views.index).JP.sum())
JP.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, JP], axis=1)
lst = [JP]
del lst 
#############################################################
KE = pd.DataFrame(page_views.groupby(page_views.index).KE.sum())
KE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KE], axis=1)
lst = [KE]
del lst 
#############################################################
KG = pd.DataFrame(page_views.groupby(page_views.index).KG.sum())
KG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KG], axis=1)
lst = [KG]
del lst 
#############################################################
KH = pd.DataFrame(page_views.groupby(page_views.index).KH.sum())
KH.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KH], axis=1)
lst = [KH]
del lst 
#############################################################
KN = pd.DataFrame(page_views.groupby(page_views.index).KN.sum())
KN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KN], axis=1)
lst = [KN]
del lst 
#############################################################
KR = pd.DataFrame(page_views.groupby(page_views.index).KR.sum())
KR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KR], axis=1)
lst = [KR]
del lst 
#############################################################
KW = pd.DataFrame(page_views.groupby(page_views.index).KW.sum())
KW.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KW], axis=1)
lst = [KW]
del lst 
#############################################################
KY = pd.DataFrame(page_views.groupby(page_views.index).KY.sum())
KY.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KY], axis=1)
lst = [KY]
del lst 
#############################################################
KZ = pd.DataFrame(page_views.groupby(page_views.index).KZ.sum())
KZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, KZ], axis=1)
lst = [KZ]
del lst 
#############################################################
LA = pd.DataFrame(page_views.groupby(page_views.index).LA.sum())
LA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, LA], axis=1)
lst = [LA]
del lst 
#############################################################
LB = pd.DataFrame(page_views.groupby(page_views.index).LB.sum())
LB.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, LB], axis=1)
lst = [LB]
del lst 
#############################################################
LK = pd.DataFrame(page_views.groupby(page_views.index).LK.sum())
LK.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, LK], axis=1)
lst = [LK]
del lst 
#############################################################
LT = pd.DataFrame(page_views.groupby(page_views.index).LT.sum())
LT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, LT], axis=1)
lst = [LT]
del lst 
#############################################################
LU = pd.DataFrame(page_views.groupby(page_views.index).LU.sum())
LU.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, LU], axis=1)
lst = [LU]
del lst 
#############################################################
LV = pd.DataFrame(page_views.groupby(page_views.index).LV.sum())
LV.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, LV], axis=1)
lst = [LV]
del lst 
#############################################################
MA = pd.DataFrame(page_views.groupby(page_views.index).MA.sum())
MA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MA], axis=1)
lst = [MA]
del lst 
#############################################################
MC = pd.DataFrame(page_views.groupby(page_views.index).MC.sum())
MC.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MC], axis=1)
lst = [MC]
del lst 
#############################################################
MD = pd.DataFrame(page_views.groupby(page_views.index).MD.sum())
MD.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MD], axis=1)
lst = [MD]
del lst 
#############################################################
ME = pd.DataFrame(page_views.groupby(page_views.index).ME.sum())
ME.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ME], axis=1)
lst = [ME]
del lst 
#############################################################
MG = pd.DataFrame(page_views.groupby(page_views.index).MG.sum())
MG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MG], axis=1)
lst = [MG]
del lst 
#############################################################
MK = pd.DataFrame(page_views.groupby(page_views.index).MK.sum())
MK.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MK], axis=1)
lst = [MK]
del lst 
#############################################################
MM = pd.DataFrame(page_views.groupby(page_views.index).MM.sum())
MM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MM], axis=1)
lst = [MM]
del lst 
#############################################################
MN = pd.DataFrame(page_views.groupby(page_views.index).MN.sum())
MN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MN], axis=1)
lst = [MN]
del lst 
#############################################################
MO = pd.DataFrame(page_views.groupby(page_views.index).MO.sum())
MO.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MO], axis=1)
lst = [MO]
del lst 
#############################################################
MR = pd.DataFrame(page_views.groupby(page_views.index).MR.sum())
MR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MR], axis=1)
lst = [MR]
del lst 
#############################################################
MT = pd.DataFrame(page_views.groupby(page_views.index).MT.sum())
MT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MT], axis=1)
lst = [MT]
del lst 
#############################################################
MU = pd.DataFrame(page_views.groupby(page_views.index).MU.sum())
MU.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MU], axis=1)
lst = [MU]
del lst 
#############################################################
MV = pd.DataFrame(page_views.groupby(page_views.index).MV.sum())
MV.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MV], axis=1)
lst = [MV]
del lst 
#############################################################
MX = pd.DataFrame(page_views.groupby(page_views.index).MX.sum())
MX.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MX], axis=1)
lst = [MX]
del lst 
#############################################################
MY = pd.DataFrame(page_views.groupby(page_views.index).MY.sum())
MY.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, MY], axis=1)
lst = [MY]
del lst 
#############################################################
NC = pd.DataFrame(page_views.groupby(page_views.index).NC.sum())
NC.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NC], axis=1)
lst = [NC]
del lst 
#############################################################
NG = pd.DataFrame(page_views.groupby(page_views.index).NG.sum())
NG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NG], axis=1)
lst = [NG]
del lst 
#############################################################
NI = pd.DataFrame(page_views.groupby(page_views.index).NI.sum())
NI.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NI], axis=1)
lst = [NI]
del lst 
#############################################################
NL = pd.DataFrame(page_views.groupby(page_views.index).NL.sum())
NL.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NL], axis=1)
lst = [NL]
del lst 
#############################################################
NO = pd.DataFrame(page_views.groupby(page_views.index).NO.sum())
NO.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NO], axis=1)
lst = [MY]
del lst 
#############################################################
NP = pd.DataFrame(page_views.groupby(page_views.index).NP.sum())
NP.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NP], axis=1)
lst = [NP]
del lst 
#############################################################
NZ = pd.DataFrame(page_views.groupby(page_views.index).NZ.sum())
NZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, NZ], axis=1)
lst = [NZ]
del lst 
#############################################################
OM = pd.DataFrame(page_views.groupby(page_views.index).OM.sum())
OM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, OM], axis=1)
lst = [OM]
del lst 
#############################################################
PA = pd.DataFrame(page_views.groupby(page_views.index).PA.sum())
PA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PA], axis=1)
lst = [PA]
del lst 
#############################################################
PE = pd.DataFrame(page_views.groupby(page_views.index).PE.sum())
PE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PE], axis=1)
lst = [PE]
del lst 
#############################################################
PG = pd.DataFrame(page_views.groupby(page_views.index).PG.sum())
PG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PG], axis=1)
lst = [PG]
del lst 
#############################################################
PH = pd.DataFrame(page_views.groupby(page_views.index).PH.sum())
PH.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PH], axis=1)
lst = [PH]
del lst 
#############################################################
PK = pd.DataFrame(page_views.groupby(page_views.index).PK.sum())
PK.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PK], axis=1)
lst = [PK]
del lst 
#############################################################
PL = pd.DataFrame(page_views.groupby(page_views.index).PL.sum())
PL.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PL], axis=1)
lst = [PL]
del lst 
#############################################################
PR = pd.DataFrame(page_views.groupby(page_views.index).PR.sum())
PR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PR], axis=1)
lst = [PR]
del lst 
#############################################################
PS = pd.DataFrame(page_views.groupby(page_views.index).PS.sum())
PS.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PS], axis=1)
lst = [PS]
#############################################################
PT = pd.DataFrame(page_views.groupby(page_views.index).PT.sum())
PT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PT], axis=1)
lst = [PT]
del lst 
#############################################################
QA = pd.DataFrame(page_views.groupby(page_views.index).QA.sum())
QA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, QA], axis=1)
lst = [QA]
del lst 
#############################################################
RE = pd.DataFrame(page_views.groupby(page_views.index).RE.sum())
RE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, RE], axis=1)
lst = [RE]
del lst 
#############################################################
RO = pd.DataFrame(page_views.groupby(page_views.index).RO.sum())
RO.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, RO], axis=1)
lst = [RO]
del lst 
#############################################################
RS = pd.DataFrame(page_views.groupby(page_views.index).RS.sum())
RS.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, RS], axis=1)
lst = [RS]
del lst 
#############################################################
RU = pd.DataFrame(page_views.groupby(page_views.index).RU.sum())
RU.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, RU], axis=1)
lst = [RU]
del lst 
#############################################################
RW = pd.DataFrame(page_views.groupby(page_views.index).RW.sum())
RW.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, RW], axis=1)
lst = [RW]
del lst 
#############################################################
SA = pd.DataFrame(page_views.groupby(page_views.index).SA.sum())
SA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SA], axis=1)
lst = [SA]
del lst
#############################################################
SC = pd.DataFrame(page_views.groupby(page_views.index).SC.sum())
SC.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SC], axis=1)
lst = [SC]
del lst 
#############################################################
SE = pd.DataFrame(page_views.groupby(page_views.index).SE.sum())
SE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SE], axis=1)
lst = [SE]
del lst 
#############################################################
SG = pd.DataFrame(page_views.groupby(page_views.index).SG.sum())
SG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SG], axis=1)
lst = [SG]
del lst 
#############################################################
SI = pd.DataFrame(page_views.groupby(page_views.index).SI.sum())
SI.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SI], axis=1)
lst = [SI]
del lst
#############################################################
SK = pd.DataFrame(page_views.groupby(page_views.index).SK.sum())
SK.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SK], axis=1)
lst = [SK]
del lst
#############################################################
SR = pd.DataFrame(page_views.groupby(page_views.index).SR.sum())
SR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SR], axis=1)
lst = [SR]
del lst
#############################################################
SV = pd.DataFrame(page_views.groupby(page_views.index).SV.sum())
SV.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SV], axis=1)
lst = [SV]
del lst
#############################################################
SX = pd.DataFrame(page_views.groupby(page_views.index).SX.sum())
SX.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, SX], axis=1)
lst = [SX]
del lst
#############################################################
TC = pd.DataFrame(page_views.groupby(page_views.index).TC.sum())
TC.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TC], axis=1)
lst = [TC]
del lst
#############################################################
TH = pd.DataFrame(page_views.groupby(page_views.index).TH.sum())
TH.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TH], axis=1)
lst = [TH]
del lst
#############################################################
TN = pd.DataFrame(page_views.groupby(page_views.index).TN.sum())
TN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TN], axis=1)
lst = [TN]
del lst
#############################################################
TR = pd.DataFrame(page_views.groupby(page_views.index).TR.sum())
TR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TR], axis=1)
lst = [TR]
del lst
#############################################################
TT = pd.DataFrame(page_views.groupby(page_views.index).TT.sum())
TT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TT], axis=1)
lst = [TT]
del lst
#############################################################
TW = pd.DataFrame(page_views.groupby(page_views.index).TW.sum())
TW.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TW], axis=1)
lst = [TW]
del lst
#############################################################
TZ = pd.DataFrame(page_views.groupby(page_views.index).TZ.sum())
TZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, TZ], axis=1)
lst = [TZ]
del lst
#############################################################
UA = pd.DataFrame(page_views.groupby(page_views.index).UA.sum())
UA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, UA], axis=1)
lst = [UA]
del lst
#############################################################
UG = pd.DataFrame(page_views.groupby(page_views.index).UG.sum())
UG.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, UG], axis=1)
lst = [UG]
del lst
#############################################################
US = pd.DataFrame(page_views.groupby(page_views.index).US.sum())
US.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, US], axis=1)
lst = [US]
del lst
#############################################################
UY = pd.DataFrame(page_views.groupby(page_views.index).UY.sum())
UY.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, UY], axis=1)
lst = [UY]
del lst
#############################################################
UZ = pd.DataFrame(page_views.groupby(page_views.index).UZ.sum())
UZ.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, UZ], axis=1)
lst = [UZ]
del lst
#############################################################
VE = pd.DataFrame(page_views.groupby(page_views.index).VE.sum())
VE.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, VE], axis=1)
lst = [VE]
del lst
#############################################################
VN = pd.DataFrame(page_views.groupby(page_views.index).VN.sum())
VN.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, VN], axis=1)
lst = [VN]
del lst
#############################################################
WS = pd.DataFrame(page_views.groupby(page_views.index).WS.sum())
WS.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, WS], axis=1)
lst = [WS]
del lst
#############################################################
ZA = pd.DataFrame(page_views.groupby(page_views.index).ZA.sum())
ZA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ZA], axis=1)
lst = [ZA]
del lst
#############################################################
ZM = pd.DataFrame(page_views.groupby(page_views.index).ZM.sum())
ZM.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ZM], axis=1)
lst = [ZM]
del lst
#############################################################
ZW = pd.DataFrame(page_views.groupby(page_views.index).ZW.sum())
ZW.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ZW], axis=1)
lst = [ZW]
del lst

The field 'mkt_medium' dennotes the type of traffic	(e.g. 'cpc', 'affiliate', 'organic', 'social'). Here we create count variables from page_views like we did with the country codes above. 

In [20]:
page_views = df[df['event_name'] == 'page_view']
page_views = page_views.set_index(['account_id'])
page_views = pd.get_dummies(page_views['mkt_medium'])
page_views = page_views.rename(columns={'3Demail': '_3Demail', '3PL': '_3PL', 'customer-story': 'customer_story',
                                        'email-signature': 'email_signature', 'guest-blog': 'guest_blog',
                                        'partner link': 'partner_link', 'press-release': 'press_release',
                                        'thankyou-page': 'thankyou_page'})
page_views.columns = ['mkt_medium_' + str(col) for col in page_views.columns]

In [21]:
_3Demail = page_views.groupby(page_views.index).mkt_medium__3Demail.sum()
_3Demail.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, _3Demail], axis=1)
lst = [_3Demail]
del lst
#############################################################
_3PL = page_views.groupby(page_views.index).mkt_medium__3PL.sum()
_3PL.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, _3PL], axis=1)
lst = [_3PL]
del lst
#############################################################
CTA = page_views.groupby(page_views.index).mkt_medium_CTA.sum()
CTA.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, CTA], axis=1)
lst = [CTA]
del lst
#############################################################
HootSuite = page_views.groupby(page_views.index).mkt_medium_HootSuite.sum()
HootSuite.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, HootSuite], axis=1)
lst = [HootSuite]
del lst
#############################################################
IncontentLink = page_views.groupby(page_views.index).mkt_medium_IncontentLink.sum()
IncontentLink.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, IncontentLink], axis=1)
lst = [IncontentLink]
del lst
#############################################################
PR = page_views.groupby(page_views.index).mkt_medium_PR.sum()
PR.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, PR], axis=1)
lst = [PR]
del lst
#############################################################
RT = page_views.groupby(page_views.index).mkt_medium_RT.sum()
RT.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, RT], axis=1)
lst = [RT]
del lst
#############################################################
adroll = page_views.groupby(page_views.index).mkt_medium_adroll.sum()
adroll.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, adroll], axis=1)
lst = [adroll]
del lst
#############################################################
appstore = page_views.groupby(page_views.index).mkt_medium_appstore.sum()
appstore.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, appstore], axis=1)
lst = [appstore]
del lst
#############################################################
banner = page_views.groupby(page_views.index).mkt_medium_banner.sum()
banner.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, banner], axis=1)
lst = [banner]
del lst
#############################################################
blog = page_views.groupby(page_views.index).mkt_medium_blog.sum()
blog.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, blog], axis=1)
lst = [blog]
del lst
#############################################################
blogrepost = page_views.groupby(page_views.index).mkt_medium_blogrepost.sum()
blogrepost.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, blogrepost], axis=1)
lst = [blogrepost]
del lst
#############################################################
cpc = page_views.groupby(page_views.index).mkt_medium_cpc.sum()
cpc.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, cpc], axis=1)
lst = [cpc]
del lst
#############################################################
customer_story = page_views.groupby(page_views.index).mkt_medium_customer_story.sum()
customer_story.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, customer_story], axis=1)
lst = [customer_story]
del lst
#############################################################
dashboard = page_views.groupby(page_views.index).mkt_medium_dashboard.sum()
dashboard.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, dashboard], axis=1)
lst = [dashboard]
del lst
#############################################################
eblast = page_views.groupby(page_views.index).mkt_medium_eblast.sum()
eblast.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, eblast], axis=1)
lst = [eblast]
del lst
#############################################################
email = page_views.groupby(page_views.index).mkt_medium_email.sum()
email.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, email], axis=1)
lst = [email]
del lst
#############################################################
email_signature = page_views.groupby(page_views.index).mkt_medium_email_signature.sum()
email_signature.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, email_signature], axis=1)
lst = [email_signature]
del lst
#############################################################
facebook = page_views.groupby(page_views.index).mkt_medium_facebook.sum()
facebook.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, facebook], axis=1)
lst = [facebook]
del lst
#############################################################
feed = page_views.groupby(page_views.index).mkt_medium_feed.sum()
feed.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, feed], axis=1)
lst = [feed]
del lst
#############################################################
forum = page_views.groupby(page_views.index).mkt_medium_forum.sum()
forum.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, forum], axis=1)
lst = [forum]
del lst
#############################################################
gp = page_views.groupby(page_views.index).mkt_medium_gp.sum()
gp.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, gp], axis=1)
lst = [gp]
del lst
#############################################################
guest_blog = page_views.groupby(page_views.index).mkt_medium_guest_blog.sum()
guest_blog.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, guest_blog], axis=1)
lst = [guest_blog]
del lst
#############################################################
guestblog = page_views.groupby(page_views.index).mkt_medium_guestblog.sum()
guestblog.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, guestblog], axis=1)
lst = [guestblog]
del lst
#############################################################
guestpost = page_views.groupby(page_views.index).mkt_medium_guestpost.sum()
guestpost.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, guestpost], axis=1)
lst = [guestpost]
del lst
#############################################################
guide = page_views.groupby(page_views.index).mkt_medium_guide.sum()
guide.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, guide], axis=1)
lst = [guide]
del lst
#############################################################
helpcenter = page_views.groupby(page_views.index).mkt_medium_helpcenter.sum()
helpcenter.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, helpcenter], axis=1)
lst = [helpcenter]
del lst
#############################################################
newsletter = page_views.groupby(page_views.index).mkt_medium_newsletter.sum()
newsletter.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, newsletter], axis=1)
lst = [newsletter]
del lst
#############################################################
notification = page_views.groupby(page_views.index).mkt_medium_notification.sum()
notification.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, notification], axis=1)
lst = [notification]
del lst
#############################################################
partner = page_views.groupby(page_views.index).mkt_medium_partner.sum()
partner.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, partner], axis=1)
lst = [partner]
del lst
#############################################################
partner_link = page_views.groupby(page_views.index).mkt_medium_partner_link.sum()
partner_link.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, partner_link], axis=1)
lst = [partner_link]
del lst
#############################################################
pdf = page_views.groupby(page_views.index).mkt_medium_pdf.sum()
pdf.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, pdf], axis=1)
lst = [pdf]
del lst
#############################################################
ppc = page_views.groupby(page_views.index).mkt_medium_ppc.sum()
ppc.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, ppc], axis=1)
lst = [ppc]
del lst
#############################################################
press_release = page_views.groupby(page_views.index).mkt_medium_press_release.sum()
press_release.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, press_release], axis=1)
lst = [press_release]
del lst
#############################################################
press_release = page_views.groupby(page_views.index).mkt_medium_press_release.sum()
press_release.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, press_release], axis=1)
lst = [press_release]
del lst
#############################################################
profile = page_views.groupby(page_views.index).mkt_medium_profile.sum()
profile.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, profile], axis=1)
lst = [profile]
del lst
#############################################################
review = page_views.groupby(page_views.index).mkt_medium_review.sum()
review.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, review], axis=1)
lst = [review]
del lst
#############################################################
reviews = page_views.groupby(page_views.index).mkt_medium_reviews.sum()
reviews.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, reviews], axis=1)
lst = [reviews]
del lst
#############################################################
shopify = page_views.groupby(page_views.index).mkt_medium_shopify.sum()
shopify.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, shopify], axis=1)
lst = [shopify]
del lst
#############################################################
social = page_views.groupby(page_views.index).mkt_medium_social.sum()
social.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, social], axis=1)
lst = [social]
del lst
#############################################################
stitchapp = page_views.groupby(page_views.index).mkt_medium_stitchapp.sum()
stitchapp.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, stitchapp], axis=1)
lst = [stitchapp]
del lst
############################################################# 
textlink = page_views.groupby(page_views.index).mkt_medium_textlink.sum()
textlink.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, textlink], axis=1)
lst = [textlink]
del lst
############################################################# 
thankyou_page = page_views.groupby(page_views.index).mkt_medium_thankyou_page.sum()
thankyou_page.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, thankyou_page], axis=1)
lst = [thankyou_page]
del lst
############################################################# 
videoLink = page_views.groupby(page_views.index).mkt_medium_videoLink.sum()
videoLink.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, videoLink], axis=1)
lst = [videoLink]
del lst
############################################################# 
web = page_views.groupby(page_views.index).mkt_medium_web.sum()
web.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, web], axis=1)
lst = [web]
del lst

In [22]:
page_views = df[df['event_name'] == 'page_view']
page_views = page_views.set_index(['account_id'])
page_views = pd.get_dummies(page_views['mkt_source'])
page_views = page_views.rename(columns={'Best-Inventory-Management-Tools': 'Best_Inventory_Management_Tools', 
'PayPal Partner Directory': 'PayPal_Partner_Directory',
'Moblized.com': 'Moblized_com',                                        
'Top-10-Order-Management-Systems': 'Top_10_Order_Management_Systems',
'Top-10-cloud-inventory-management-software': 'Top_10_cloud_inventory_management_software',
'adroll.com': 'adroll_com', 
'source_company_x.com': 'source_company_x_com',                                        
'bigcommerce.com': 'bigcommerce_com',          
'capterra.com': 'capterra_com',
'company_x.com?utm_medium=blog?utm_campaign=tax-season': 'company_x_com_utm_medium_blog_utm_campaign_tax_season',
'dbny.co': 'dbny_co',
'etsy.com': 'etsy_com',
'facebook.com': 'facebook_com',         
'getapp.com': 'getapp_com',
'google.com': 'google_com',
'indinero.com': 'indinero_com',
'interface-text': 'interface_text',
'intuit.com': 'intuit_com',
'justuno.com': 'justuno_com',  
'lightspeed.com': 'lightspeed_com',
'linkedin.com': 'linkedin_com',
'magento.com': 'magento_com',
'merchantmaverick.com': 'merchantmaverick_com',                                   
'pec-newsletter': 'pec_newsletter',
'pixafy.com': 'pixafy_com' ,
'reddit.com': 'reddit_com',
'scoutsft.com': 'scoutsft_com',
'sellercentral.amazon.com': 'sellercentral_amazon_com',               
'shipstation.com': 'shipstation_com',
'shipstation.com?utm_medium=blog?utm_campaign=future-of-commercemedium=partner': 'shipstation_com_utm_medium_blog',
'shipwire.com': 'shipwire_com',                                       
'shopify.com': 'shopify_com',              
'squareup.com': 'squareup_com',
'squareup.com.ca': 'squareup_com_ca',
'squareup.com.global': 'squareup_com_global',                                        
'twitter.com': 'twitter_com',
'vendhq.com': 'vendhq_com',                                       
'wiser.com': 'wiser_com',
'woocommerce.com': 'woocommerce_com',
'woothemes.com': 'woothemes_com',
'xero.com': 'xero_com',
'youtube.com': 'youtube_com'})
page_views.columns = ['mkt_source_' + str(col) for col in page_views.columns]
page_views = page_views.rename(columns={'mkt_source_company_x.com': 'mkt_source_company_x_com'})

In [23]:
mkt_source_Best_Inventory_Management_Tools = \
page_views.groupby(page_views.index).mkt_source_Best_Inventory_Management_Tools.sum()
mkt_source_Best_Inventory_Management_Tools.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Best_Inventory_Management_Tools], axis=1)
lst = [mkt_source_Best_Inventory_Management_Tools]
del lst
############################################################# 
mkt_source_Desk = page_views.groupby(page_views.index).mkt_source_Desk.sum()
mkt_source_Desk.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Desk], axis=1)
lst = [mkt_source_Desk]
del lst
############################################################# 
mkt_source_FBButton = page_views.groupby(page_views.index).mkt_source_FBButton.sum()
mkt_source_FBButton.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_FBButton], axis=1)
lst = [mkt_source_FBButton]
del lst
############################################################# 
mkt_source_Gainsight = page_views.groupby(page_views.index).mkt_source_Gainsight.sum()
mkt_source_Gainsight.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Gainsight], axis=1)
lst = [mkt_source_Gainsight]
del lst
############################################################# 
mkt_source_Moblized_com = page_views.groupby(page_views.index).mkt_source_Moblized_com.sum()
mkt_source_Moblized_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Moblized_com], axis=1)
lst = [mkt_source_Moblized_com]
del lst
############################################################# 
mkt_source_PayPal_Partner_Directory = \
page_views.groupby(page_views.index).mkt_source_PayPal_Partner_Directory.sum()
mkt_source_PayPal_Partner_Directory.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_PayPal_Partner_Directory], axis=1)
lst = [mkt_source_PayPal_Partner_Directory]
del lst
############################################################# 
mkt_source_PayPal_Partner_Directory = \
page_views.groupby(page_views.index).mkt_source_PayPal_Partner_Directory.sum()
mkt_source_PayPal_Partner_Directory.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_PayPal_Partner_Directory], axis=1)
lst = [mkt_source_PayPal_Partner_Directory]
del lst
############################################################# 
mkt_source_SellerEngine = \
page_views.groupby(page_views.index).mkt_source_SellerEngine.sum()
mkt_source_SellerEngine.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_SellerEngine], axis=1)
lst = [mkt_source_SellerEngine]
del lst
############################################################# 
mkt_source_Success = \
page_views.groupby(page_views.index).mkt_source_Success.sum()
mkt_source_Success.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Success], axis=1)
lst = [mkt_source_Success]
del lst
############################################################# 
mkt_source_Top_10_Order_Management_Systems = \
page_views.groupby(page_views.index).mkt_source_Top_10_Order_Management_Systems.sum()
mkt_source_Top_10_Order_Management_Systems.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Top_10_Order_Management_Systems], axis=1)
lst = [mkt_source_Top_10_Order_Management_Systems]
del lst
############################################################# 
mkt_source_Top_10_cloud_inventory_management_software = \
page_views.groupby(page_views.index).mkt_source_Top_10_cloud_inventory_management_software.sum()
mkt_source_Top_10_cloud_inventory_management_software.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Top_10_cloud_inventory_management_software], axis=1)
lst = [mkt_source_Top_10_cloud_inventory_management_software]
del lst
############################################################# 
mkt_source_Twitter = \
page_views.groupby(page_views.index).mkt_source_Twitter.sum()
mkt_source_Twitter.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_Twitter], axis=1)
lst = [mkt_source_Twitter]
del lst
############################################################# 
mkt_source_adroll = \
page_views.groupby(page_views.index).mkt_source_adroll.sum()
mkt_source_adroll.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_adroll], axis=1)
lst = [mkt_source_adroll]
del lst
#############################################################
mkt_source_adroll_com = \
page_views.groupby(page_views.index).mkt_source_adroll_com.sum()
mkt_source_adroll_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_adroll_com], axis=1)
lst = [mkt_source_adroll_com]
del lst
############################################################# 
mkt_source_amazon = \
page_views.groupby(page_views.index).mkt_source_amazon.sum()
mkt_source_amazon.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_amazon], axis=1)
lst = [mkt_source_amazon]
del lst
#############################################################
mkt_source_appeagle = \
page_views.groupby(page_views.index).mkt_source_appeagle.sum()
mkt_source_appeagle.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_appeagle], axis=1)
lst = [mkt_source_appeagle]
del lst
#############################################################
mkt_source_bigcommerce_com = \
page_views.groupby(page_views.index).mkt_source_bigcommerce_com.sum()
mkt_source_bigcommerce_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_bigcommerce_com], axis=1)
lst = [mkt_source_bigcommerce_com]
del lst
#############################################################
mkt_source_blog = \
page_views.groupby(page_views.index).mkt_source_blog.sum()
mkt_source_blog.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_blog], axis=1)
lst = [mkt_source_blog]
del lst
#############################################################
mkt_source_capterra_com = \
page_views.groupby(page_views.index).mkt_source_capterra_com.sum()
mkt_source_capterra_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_capterra_com], axis=1)
lst = [mkt_source_capterra_com]
del lst
#############################################################
mkt_source_cartdesigners = \
page_views.groupby(page_views.index).mkt_source_cartdesigners.sum()
mkt_source_cartdesigners.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_cartdesigners], axis=1)
lst = [mkt_source_cartdesigners]
del lst
#############################################################
mkt_source_company_x = \
page_views.groupby(page_views.index).mkt_source_company_x.sum()
mkt_source_company_x.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_company_x], axis=1)
lst = [mkt_source_company_x]
del lst
#############################################################
mkt_source_company_x_com = \
page_views.groupby(page_views.index).mkt_source_company_x_com.sum()
mkt_source_company_x_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_company_x_com], axis=1)
lst = [mkt_source_company_x_com]
del lst
#############################################################
mkt_source_company_x_com_utm_medium_blog_utm_campaign_tax_season = \
page_views.groupby(page_views.index).mkt_source_company_x_com_utm_medium_blog_utm_campaign_tax_season.sum()
mkt_source_company_x_com_utm_medium_blog_utm_campaign_tax_season.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_company_x_com_utm_medium_blog_utm_campaign_tax_season], axis=1)
lst = [mkt_source_company_x_com_utm_medium_blog_utm_campaign_tax_season]
del lst
#############################################################
mkt_source_content = \
page_views.groupby(page_views.index).mkt_source_content.sum()
mkt_source_content.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_content], axis=1)
lst = [mkt_source_content]
del lst
#############################################################
mkt_source_dbny_co = \
page_views.groupby(page_views.index).mkt_source_dbny_co.sum()
mkt_source_dbny_co.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_dbny_co], axis=1)
lst = [mkt_source_dbny_co]
del lst
#############################################################
mkt_source_designsponge = \
page_views.groupby(page_views.index).mkt_source_designsponge.sum()
mkt_source_designsponge.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_designsponge], axis=1)
lst = [mkt_source_designsponge]
del lst
#############################################################
mkt_source_ebay = \
page_views.groupby(page_views.index).mkt_source_ebay.sum()
mkt_source_ebay.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_ebay], axis=1)
lst = [mkt_source_ebay]
del lst
#############################################################
mkt_source_ecommby = \
page_views.groupby(page_views.index).mkt_source_ecommby.sum()
mkt_source_ecommby.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_ecommby], axis=1)
lst = [mkt_source_ecommby]
del lst
#############################################################
mkt_source_etsy_com = \
page_views.groupby(page_views.index).mkt_source_etsy_com.sum()
mkt_source_etsy_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_etsy_com], axis=1)
lst = [mkt_source_etsy_com]
del lst
#############################################################
mkt_source_exacttarget = \
page_views.groupby(page_views.index).mkt_source_exacttarget.sum()
mkt_source_exacttarget.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_exacttarget], axis=1)
lst = [mkt_source_exacttarget]
del lst
#############################################################
mkt_source_facebook = \
page_views.groupby(page_views.index).mkt_source_facebook.sum()
mkt_source_facebook.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_facebook], axis=1)
lst = [mkt_source_facebook]
del lst
#############################################################
mkt_source_facebook_com = \
page_views.groupby(page_views.index).mkt_source_facebook_com.sum()
mkt_source_facebook_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_facebook_com], axis=1)
lst = [mkt_source_facebook_com]
del lst
#############################################################
mkt_source_feedburner = \
page_views.groupby(page_views.index).mkt_source_feedburner.sum()
mkt_source_feedburner.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_feedburner], axis=1)
lst = [mkt_source_feedburner]
del lst
#############################################################
mkt_source_forum = \
page_views.groupby(page_views.index).mkt_source_forum.sum()
mkt_source_forum.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_forum], axis=1)
lst = [mkt_source_forum]
del lst
#############################################################
mkt_source_getapp = \
page_views.groupby(page_views.index).mkt_source_getapp.sum()
mkt_source_getapp.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_getapp], axis=1)
lst = [mkt_source_getapp]
del lst
#############################################################
mkt_source_getapp_com = \
page_views.groupby(page_views.index).mkt_source_getapp_com.sum()
mkt_source_getapp_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_getapp_com], axis=1)
lst = [mkt_source_getapp_com]
del lst
#############################################################
mkt_source_google = \
page_views.groupby(page_views.index).mkt_source_google.sum()
mkt_source_google.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_google], axis=1)
lst = [mkt_source_google]
del lst
#############################################################
mkt_source_google_com = \
page_views.groupby(page_views.index).mkt_source_google_com.sum()
mkt_source_google_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_google_com], axis=1)
lst = [mkt_source_google_com]
del lst
#############################################################
mkt_source_hs_automation = \
page_views.groupby(page_views.index).mkt_source_hs_automation.sum()
mkt_source_hs_automation.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_hs_automation], axis=1)
lst = [mkt_source_hs_automation]
del lst
#############################################################
mkt_source_hs_email = \
page_views.groupby(page_views.index).mkt_source_hs_email.sum()
mkt_source_hs_email.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_hs_email], axis=1)
lst = [mkt_source_hs_email]
del lst
#############################################################
mkt_source_imadeitmarket = \
page_views.groupby(page_views.index).mkt_source_imadeitmarket.sum()
mkt_source_imadeitmarket.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_imadeitmarket], axis=1)
lst = [mkt_source_imadeitmarket]
del lst
#############################################################
mkt_source_indinero_com = \
page_views.groupby(page_views.index).mkt_source_indinero_com.sum()
mkt_source_indinero_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_indinero_com], axis=1)
lst = [mkt_source_indinero_com]
del lst
#############################################################
mkt_source_interface_text = \
page_views.groupby(page_views.index).mkt_source_interface_text.sum()
mkt_source_interface_text.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_interface_text], axis=1)
lst = [mkt_source_interface_text]
del lst
#############################################################
mkt_source_intuit_com = \
page_views.groupby(page_views.index).mkt_source_intuit_com.sum()
mkt_source_intuit_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_intuit_com], axis=1)
lst = [mkt_source_intuit_com]
del lst
#############################################################
mkt_source_justuno_com = \
page_views.groupby(page_views.index).mkt_source_justuno_com.sum()
mkt_source_justuno_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_justuno_com], axis=1)
lst = [mkt_source_justuno_com]
del lst
#############################################################
mkt_source_kabbage = \
page_views.groupby(page_views.index).mkt_source_kabbage.sum()
mkt_source_kabbage.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_kabbage], axis=1)
lst = [mkt_source_kabbage]
del lst
#############################################################
mkt_source_lightspeed_com = \
page_views.groupby(page_views.index).mkt_source_lightspeed_com.sum()
mkt_source_lightspeed_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_lightspeed_com], axis=1)
lst = [mkt_source_lightspeed_com]
del lst
#############################################################
mkt_source_linkedin = \
page_views.groupby(page_views.index).mkt_source_linkedin.sum()
mkt_source_linkedin.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_linkedin], axis=1)
lst = [mkt_source_linkedin]
del lst
#############################################################
mkt_source_linkedin_com = \
page_views.groupby(page_views.index).mkt_source_linkedin_com.sum()
mkt_source_linkedin_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_linkedin_com], axis=1)
lst = [mkt_source_linkedin_com]
del lst
#############################################################
mkt_source_magento_com = \
page_views.groupby(page_views.index).mkt_source_magento_com.sum()
mkt_source_magento_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_magento_com], axis=1)
lst = [mkt_source_magento_com]
del lst
#############################################################
mkt_source_merchantmaverick_com = \
page_views.groupby(page_views.index).mkt_source_merchantmaverick_com.sum()
mkt_source_merchantmaverick_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_merchantmaverick_com], axis=1)
lst = [mkt_source_merchantmaverick_com]
del lst
#############################################################
mkt_source_mogulmom = \
page_views.groupby(page_views.index).mkt_source_mogulmom.sum()
mkt_source_mogulmom.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_mogulmom], axis=1)
lst = [mkt_source_mogulmom]
del lst
#############################################################
mkt_source_pec_newsletter = \
page_views.groupby(page_views.index).mkt_source_pec_newsletter.sum()
mkt_source_pec_newsletter.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_pec_newsletter], axis=1)
lst = [mkt_source_pec_newsletter]
del lst
#############################################################
mkt_source_pixafy_com = \
page_views.groupby(page_views.index).mkt_source_pixafy_com.sum()
mkt_source_pixafy_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_pixafy_com], axis=1)
lst = [mkt_source_pixafy_com]
del lst
#############################################################
mkt_source_practicalecommerce = \
page_views.groupby(page_views.index).mkt_source_practicalecommerce.sum()
mkt_source_practicalecommerce.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_practicalecommerce], axis=1)
lst = [mkt_source_practicalecommerce]
del lst
#############################################################
mkt_source_prnewswire = \
page_views.groupby(page_views.index).mkt_source_prnewswire.sum()
mkt_source_prnewswire.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_prnewswire], axis=1)
lst = [mkt_source_prnewswire]
del lst
#############################################################
mkt_source_reddit_com = \
page_views.groupby(page_views.index).mkt_source_reddit_com.sum()
mkt_source_reddit_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_reddit_com], axis=1)
lst = [mkt_source_reddit_com]
del lst
#############################################################
mkt_source_scoutsft_com = \
page_views.groupby(page_views.index).mkt_source_scoutsft_com.sum()
mkt_source_scoutsft_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_scoutsft_com], axis=1)
lst = [mkt_source_scoutsft_com]
del lst
#############################################################
mkt_source_sellercentral_amazon_com = \
page_views.groupby(page_views.index).mkt_source_sellercentral_amazon_com.sum()
mkt_source_sellercentral_amazon_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_sellercentral_amazon_com], axis=1)
lst = [mkt_source_sellercentral_amazon_com]
del lst
#############################################################
mkt_source_shipstation = \
page_views.groupby(page_views.index).mkt_source_shipstation.sum()
mkt_source_shipstation.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shipstation], axis=1)
lst = [mkt_source_shipstation]
del lst
#############################################################
mkt_source_shipstation_com = \
page_views.groupby(page_views.index).mkt_source_shipstation_com.sum()
mkt_source_shipstation_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shipstation_com], axis=1)
lst = [mkt_source_shipstation_com]
del lst
#############################################################
mkt_source_shipstation_com_utm_medium_blog = \
page_views.groupby(page_views.index).mkt_source_shipstation_com_utm_medium_blog.sum()
mkt_source_shipstation_com_utm_medium_blog.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shipstation_com_utm_medium_blog], axis=1)
lst = [mkt_source_shipstation_com_utm_medium_blog]
del lst
#############################################################
mkt_source_shipwire_com = \
page_views.groupby(page_views.index).mkt_source_shipwire_com.sum()
mkt_source_shipwire_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shipwire_com], axis=1)
lst = [mkt_source_shipwire_com]
del lst
#############################################################
mkt_source_shoeboxed = \
page_views.groupby(page_views.index).mkt_source_shoeboxed.sum()
mkt_source_shoeboxed.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shoeboxed], axis=1)
lst = [mkt_source_shoeboxed]
del lst
#############################################################
mkt_source_shopify = \
page_views.groupby(page_views.index).mkt_source_shopify.sum()
mkt_source_shopify.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shopify], axis=1)
lst = [mkt_source_shopify]
del lst
#############################################################
mkt_source_shopify_com = \
page_views.groupby(page_views.index).mkt_source_shopify_com.sum()
mkt_source_shopify_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_shopify_com], axis=1)
lst = [mkt_source_shopify_com]
del lst
#############################################################
mkt_source_social = \
page_views.groupby(page_views.index).mkt_source_social.sum()
mkt_source_social.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_social], axis=1)
lst = [mkt_source_social]
del lst
#############################################################
mkt_source_square = \
page_views.groupby(page_views.index).mkt_source_square.sum()
mkt_source_square.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_square], axis=1)
lst = [mkt_source_square]
del lst
#############################################################
mkt_source_squareup_com = \
page_views.groupby(page_views.index).mkt_source_squareup_com.sum()
mkt_source_squareup_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_squareup_com], axis=1)
lst = [mkt_source_squareup_com]
del lst
#############################################################
mkt_source_squareup_com_ca = \
page_views.groupby(page_views.index).mkt_source_squareup_com_ca.sum()
mkt_source_squareup_com_ca.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_squareup_com_ca], axis=1)
lst = [mkt_source_squareup_com_ca]
del lst
#############################################################
mkt_source_squareup_com_global = \
page_views.groupby(page_views.index).mkt_source_squareup_com_global.sum()
mkt_source_squareup_com_global.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_squareup_com_global], axis=1)
lst = [mkt_source_squareup_com_global]
del lst
#############################################################
mkt_source_stitchapp = \
page_views.groupby(page_views.index).mkt_source_stitchapp.sum()
mkt_source_stitchapp.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_stitchapp], axis=1)
lst = [mkt_source_stitchapp]
del lst
#############################################################
mkt_source_success = \
page_views.groupby(page_views.index).mkt_source_success.sum()
mkt_source_success.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_success], axis=1)
lst = [mkt_source_success]
del lst
#############################################################
mkt_source_twitter = \
page_views.groupby(page_views.index).mkt_source_twitter.sum()
mkt_source_twitter.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_twitter], axis=1)
lst = [mkt_source_twitter]
del lst
#############################################################
mkt_source_twitter_com = \
page_views.groupby(page_views.index).mkt_source_twitter_com.sum()
mkt_source_twitter_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_twitter_com], axis=1)
lst = [mkt_source_twitter_com]
del lst
#############################################################
mkt_source_vendhq_com = \
page_views.groupby(page_views.index).mkt_source_vendhq_com.sum()
mkt_source_vendhq_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_vendhq_com], axis=1)
lst = [mkt_source_vendhq_com]
del lst
#############################################################
mkt_source_vero = \
page_views.groupby(page_views.index).mkt_source_vero.sum()
mkt_source_vero.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_vero], axis=1)
lst = [mkt_source_vero]
del lst
#############################################################
mkt_source_weeklyemail = \
page_views.groupby(page_views.index).mkt_source_weeklyemail.sum()
mkt_source_weeklyemail.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_weeklyemail], axis=1)
lst = [mkt_source_weeklyemail]
del lst
#############################################################
mkt_source_wiser_com = \
page_views.groupby(page_views.index).mkt_source_wiser_com.sum()
mkt_source_wiser_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_wiser_com], axis=1)
lst = [mkt_source_wiser_com]
del lst
#############################################################
mkt_source_woocommerce = \
page_views.groupby(page_views.index).mkt_source_woocommerce.sum()
mkt_source_woocommerce.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_woocommerce], axis=1)
lst = [mkt_source_woocommerce]
del lst
#############################################################
mkt_source_woocommerce_com = \
page_views.groupby(page_views.index).mkt_source_woocommerce_com.sum()
mkt_source_woocommerce_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_woocommerce_com], axis=1)
lst = [mkt_source_woocommerce_com]
del lst
#############################################################
mkt_source_woothemes_com = \
page_views.groupby(page_views.index).mkt_source_woothemes_com.sum()
mkt_source_woothemes_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_woothemes_com], axis=1)
lst = [mkt_source_woothemes_com]
del lst
#############################################################
mkt_source_xero = \
page_views.groupby(page_views.index).mkt_source_xero.sum()
mkt_source_xero.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_xero], axis=1)
lst = [mkt_source_xero]
del lst
#############################################################
mkt_source_xero_com = \
page_views.groupby(page_views.index).mkt_source_xero_com.sum()
mkt_source_xero_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_xero_com], axis=1)
lst = [mkt_source_xero_com]
del lst
#############################################################
mkt_source_youtube_com = \
page_views.groupby(page_views.index).mkt_source_youtube_com.sum()
mkt_source_youtube_com.fillna(value=0, inplace=True)
event_df = pd.concat([event_df, mkt_source_youtube_com], axis=1)
lst = [mkt_source_youtube_com]
del lst

In [24]:
df["cc"] = df["cc_date_added"].notnull()
CCs = pd.DataFrame(df.groupby(df["account_id"]).cc.sum())
CCs.ix[CCs.cc > 0, 'cc'] = 1
event_df = pd.concat([event_df, CCs], axis=1)

Zero fill the munged data object. 

In [25]:
event_df.fillna(value=0, inplace=True)

Export the munged data object. 

In [26]:
event_df.to_csv('munged_df.csv')